### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées dans un tableau de bord permettant d'avoir un portrait d'ensemble des données.  

---

In [1]:
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import dash_bootstrap_components as dbc
from utils.SADVR_utils import *


app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Charger les données
data = updateInfoProfs()
expertises = data[['idsadvr', 'affiliations', 'etablissementsAffilies', 'expertise']]
demographics = data[['idsadvr', 'sexe', 'langues', 'formations', 'affiliations']]

# Normalisation - donnée expertises
# expertises = data[['idsadvr', 'affiliations', 'etablissementsAffilies', 'expertise']]
# toNormalize = ['affiliations', 'etablissementsAffilies', 'expertise', 'expertise.secteursRecherche',
#                 'expertise.disciplines', 'expertise.pays', 'expertise.continents', 'expertise.periodesChronologiques']

# for c in toNormalize:
#     expertises = explodeNormalize(expertises, c)

# drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble', 'affiliations.fonction.codeSad', 
#         'affiliations.fonction.nom', 'affiliations.local', 'affiliations.exclusion', 'affiliations.exclusionTel',
#         'affiliations.uniteAdministrative.codeSad', 'affiliations.uniteAdministrative.nom', 'affiliations.telephone.numero',
#         'affiliations.telephone.poste', 'expertise.phraseCle']

# expertises = expertises.drop(columns=drop)

# Normalisation - données demographiques
fonctionsProf = pd.read_csv('utils/fonctionsProfs.csv')['codeSad'].tolist()
toNormalize = ['langues', 'affiliations', 'formations', 'formations.disciplines', 'formations.institutions']
for c in toNormalize:
    demographics = explodeNormalize(demographics, c)

columns = pd.read_csv('utils/columnsDemographics.csv')['columns'].tolist()
demographics = demographics[[x for x in demographics.columns if x in columns]]

demographics = demographics[demographics['affiliations.fonction.codeSad'].isin(fonctionsProf)]
demographics.to_csv('tables/demographics.csv', index=False)


### 1e visuel : 4 mesures "Count"
# 1: nombre de professeurs
nbProfs = len(data.drop_duplicates(subset='idsadvr'))

# Visuel : card
cardNbProfs = dbc.Card(
    [
        dbc.CardBody(
            [
                html.H4(str(nbProfs), className="card-title"),
                html.P("Professeur·e·s",
                    className="card-text",
                ),
            ],
            style={'textAlign':'center'}
        ),
    ],
)


# 2: nombre de facultés et écoles
nbFacultes = len(getTable('facultes'))
# Visuel : card
cardNbFacultes = dbc.Card(
    [
        dbc.CardBody(
            [
                html.H4(str(nbFacultes), className="card-title"),
                html.P("Facultés et écoles",
                    className="card-text",
                ),
            ],
            style={'textAlign':'center'}
        ),
    ]
)

# 3: nombre de départements
nbDepartements = len(getTable('departements'))

# Visuel : card
cardNbDepartements = dbc.Card(
    [
        dbc.CardBody(
            [
                html.H4(str(nbProfs), className="card-title"),
                html.P("Départements",
                    className="card-text")
            ],
            style={'textAlign':'center'}
        ),
    ],
)

# 4: Nombre d'établissements affiliés
# etablissementsAffilies = expertises.dropna(subset='etablissementsAffilies.nom')
# etablissementsAffilies = etablissementsAffilies.drop_duplicates(subset=(['idsadvr', 'etablissementsAffilies.nom']))
# etablissementsAffilies = pd.DataFrame(plotVariable(etablissementsAffilies, 'etablissementsAffilies.nom'))
# nbEtablissementsAffilies = len(etablissementsAffilies)

nbEtablissementsAffilies = 27

# Visuel: Card
cardNbEtablissementsAffilies = dbc.Card(
    [
        dbc.CardBody(
            [
                html.H4(str(nbEtablissementsAffilies), className="card-title"),
                html.P("Établissements affiliés",
                    className="card-text")
            ],
            style={'textAlign':'center'}
        ),
    ],
)

# Visuel "Cartes" - style
shadow = '2px 2px 6px lightgrey'

# Genre
mapping = {'M': 'Hommes', 'F': 'Femmes', 'A': 'Autres'}
genre = plotVariable(demographics, 'sexe', mapping=mapping)

figGenre = px.pie(
    names = genre['labels'],
    values = genre['count'],
    hole=0.5,
    title='Identité de genre',
    width=500,
    #color_discrete_sequence= px.colors.qualitative.Antique   
)

# Pays d'obtention du dernier diplôme
paysFormation = pd.DataFrame(plotVariable(demographics, 'formations.institutions.paysNom'))
paysFormation = groupOtherValues(paysFormation, 8)

order = [x for x in paysFormation['labels'] if not (x == 'Autre')] + ['Autre']

figPaysDiplome = px.pie(
    paysFormation, 
    values= paysFormation['count'], 
    names= paysFormation['labels'], 
    title="Lieu d'obtention du dernier diplôme",
    hole=0.5,
    width=600, 
    category_orders= {'labels':
        order
    }
)

In [2]:
app.layout = html.Div(
    children = [html.H1
        (
            "SADVR - Portrait statistique",
            style = {
                'textAlign': 'center',
                'color': 'white', 
                'backgroundColor': '#0b113a',
                'padding': '30px',
                'fontFamily': 'Calibri'
                }
        ),
        
        html.Div(
            style = {
                'border' : '1px solid lightgrey',
                'paddingTop': '2%',
                'marginBottom': '2%',
                'marginLeft': '10%',
                'marginRight': '10%',
                'paddingLeft': '5%',
                'paddingRight': '5%',
                'boxShadow': shadow
            }, 
            children= [
                # Titre de l'onglet
                html.H2(
                    'Professeur·e·s - Portrait sociodémographique', 
                    style={
                            'textAlign': 'left',
                            'color': '#444444',
                    }
                ),

                # Trait de soulignement
                html.Hr(
                    style={
                        'borderTop': '4px solid #52B782',
                        'width' : '50%',
                        'marginBottom': '50px'
                    }
                ),

                # Cartes
                html.Div(
                    [
                        dbc.Row(
                            [
                                dbc.Col(
                                    cardNbProfs,
                                    width = 6,
                                    lg = 3  
                                ),
                                dbc.Col(
                                    cardNbFacultes,
                                    width = 6,
                                    lg = 3   
                                ),
                                dbc.Col(
                                    cardNbDepartements,
                                    width = 6,
                                    lg = 3  
                                ),
                                
                                dbc.Col(
                                    cardNbEtablissementsAffilies, 
                                    width = 6,
                                    lg = 3  
                                ),
                            ]
                        )
                    ]
                ),

                # Trait de soulignement
                html.Hr(
                    style={
                        'borderTop': '1px solid #lightgrey',
                        'marginTop': '50px',
                        'marginBottom': '50px'
                    }
                ),

                # Secteurs - genre + pays d'obtention du dernier diplôme
                html.Div(
                    [dbc.Row
                        (
                            [
                                dbc.Col(
                                    dcc.Graph(figure=figGenre),
                                    width = 6,
                                ),
                                dbc.Col(
                                    dcc.Graph(figure=figPaysDiplome),
                                    width = 6,
                                )
                            ]
                        ), 
                    ],
                ),

                # Trait de soulignement
                html.Hr(
                    style={
                        'borderTop': '1px solid #lightgrey',
                        'marginBottom': '50px'
                    }
                ),
            ],
        )
    ]
)

if __name__ == '__main__':
    app.run(debug=True)
